## model
- BRISQUE
- hyperiqa
- ilniqe
- niqe

In [2]:
import glob
import os
from pyiqa import create_metric
from tqdm import tqdm
import csv

def main():
    # Manually set arguments
    input_path = r'C:\Users\(Ai)AiSukmoren\Desktop\slim-ChatGPT\pics'
    ref_path = None  #r'C:\Users\(Ai)AiSukmoren\Desktop\slim-ChatGPT\pics'  # set this if you use FR metrics
    metric_mode = 'NR'  # or 'FR' if you have a reference
    metric_name = 'TRES'
    save_file = 'quality_assessment_results.csv'

    metric_name = metric_name.lower()

    # set up IQA model
    iqa_model = create_metric(metric_name, metric_mode=metric_mode)
    metric_mode = iqa_model.metric_mode

    if os.path.isfile(input_path):
        input_paths = [input_path]
        if ref_path is not None:
            ref_paths = [ref_path]
    else:
        input_paths = sorted(glob.glob(os.path.join(input_path, '*')))
        if ref_path is not None:
            ref_paths = sorted(glob.glob(os.path.join(input_path, '*')))
    # if save file contain file path 
    if save_file:
        sf = open(save_file, 'w')
        sfwriter = csv.writer(sf)

    avg_score = 0
    test_img_num = len(input_paths)
    if metric_name != 'fid':
        pbar = tqdm(total=test_img_num, unit='image')
        for idx, img_path in enumerate(input_paths):
            img_name = os.path.basename(img_path)
            if metric_mode == 'FR':
                ref_img_path = ref_paths[idx]
            else:
                ref_img_path = None

            score = iqa_model(img_path, ref_img_path).cpu().item()
            avg_score += score
            pbar.update(1)
            pbar.set_description(f'{metric_name} of {img_name}: {score}')
            pbar.write(f'{metric_name} of {img_name}: {score}')
            if save_file:
                sfwriter.writerow([img_name, score])
            
        pbar.close()
        avg_score /= test_img_num
    else:
        assert os.path.isdir(input_path), 'input path must be a folder for FID.'
        avg_score = iqa_model(input_path, ref_path)
    
    msg = f'Average {metric_name} score of {input_path} with {test_img_num} images is: {avg_score}'
    print(msg)
    if save_file:
        sf.close()

    if save_file:
        print(f'Done! Results are in {save_file}.')
    else:
        print(f'Done!')


if __name__ == '__main__':
    main()

Loading pretrained model TReS from C:\Users\(Ai)AiSukmoren\.cache\torch\hub\checkpoints\tres_koniq-f0502926.pth


tres of 000019.jpg: 74.67933654785156:  50%|█████     | 1/2 [00:32<00:32, 32.20s/image]

tres of 000019.jpg: 74.67933654785156


tres of 000020.jpg: 67.10346984863281: 100%|██████████| 2/2 [01:04<00:00, 32.18s/image]

tres of 000020.jpg: 67.10346984863281
Average tres score of C:\Users\(Ai)AiSukmoren\Desktop\slim-ChatGPT\pics with 2 images is: 70.89140319824219
Done! Results are in quality_assessment_results.csv.
